LỚP 19CLC5 - NHẬP MÔN AI
* 19127097 - NGUYỄN NGỌC PHƯƠNG ANH
* 19127078 - NGUYỄN ĐỖ THANH TRÚC
* 19127142 - TRẦN THÁI ĐỨC HIẾU

Unzip file dữ liệu từ kaggle gồm 25000 ảnh chó mèo bằng cách tham gia cuộc thi dogs-vs-cats

In [1]:
import os
base_dir = "../input/dogs-vs-cats/"

train_link = os.path.join(base_dir, "train.zip")
test_link = os.path.join(base_dir, "test1.zip")

# ../input/dogs-vs-cats-redux-kernels-edition/test.zip
import zipfile
with zipfile.ZipFile(train_link,"r") as z:
 z.extractall()
with zipfile.ZipFile(test_link,"r") as z:
 z.extractall()


FileNotFoundError: [Errno 2] No such file or directory: '../input/dogs-vs-cats/train.zip'

BẮT ĐẦU CHẠY TỪ ĐÂY: Import các thư viện

In [ ]:
#Bước 1: Import thư viện
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dropout,Flatten,Dense,Activation, BatchNormalization
from sklearn.model_selection import train_test_split

In [ ]:
#Bước 2: Xác định các thuộc tính hình ảnh - Kích thước, chiều
Image_Channels=3
Image_Width=224
Image_Height=224
Image_Size=(Image_Width,Image_Height)

train_link  = "../working/train"
test_link  = "../working/test1"

Đánh label cho dữ liệu, 0 là mèo; 1 là chó


In [ ]:
# Bước 3: Chuẩn bị bộ dữ liệu
filenames=os.listdir(train_link)

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})


Callbacks dừng lại sau 4 vòng với val_accuracy tăng quá ít và lưu lại model tốt nhất

In [ ]:
#Bước 4: Định nghĩa callbacks và learning rate
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_accuracy', 
    mode = 'auto',
    verbose = 1, 
    patience = 4
    )
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = "checkpoint.h5",
    monitor = 'val_accuracy', 
    save_best_only = True, 
    mode = 'auto'
    )

callbacks = [early_stopping, model_checkpoint]

Hàm tách dự liệu ra thành validation set và train set, tỉ lệ 1:4

In [ ]:
# Bước 5: Quản lý dữ liệu - chia tập vali và train
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,test_size=0.25,
  random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

batch_size=32 


Điều chỉnh độ chính xác, cho phép xoay ảnh, rescale, zoom, etc; resize ảnh về ImageSize

In [ ]:
#Bước 6: Generate tập dữ liệu và điều chỉnh
train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_dataframe(train_df,train_link,x_col='filename',y_col='category',class_mode='binary', batch_size=batch_size, target_size=Image_Size)
validation_generator = val_datagen.flow_from_dataframe(validate_df,train_link,x_col='filename',y_col='category',class_mode='binary', batch_size=batch_size, target_size=Image_Size)


In [ ]:
#Bước 7: Build model khi CHƯA CÓ MODEL (không load model)


model = Sequential()

model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv1', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block1_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block1_maxpool'))

model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv1'))
model.add(Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block2_conv2'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block2_maxpool'))

model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv1'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv2'))
model.add(Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block3_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block3_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block4_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block4_maxpool'))

model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv1'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv2'))
model.add(Conv2D(512, (3,3), activation="relu", padding="same", kernel_initializer='he_uniform', name='block5_conv3'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), name='block5_maxpool'))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
#Bước 8: compile và đánh giá - KHÔNG CHẠY KHI DÙNG MODEL LOAD LẠI
opt=tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9) 

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
#Bước 9: Fit model (training) - CHẠY KHI LOAD MODEL để kiểm tra accuracy hiện tại rồi tắt đi
history = model.fit_generator(generator=train_generator, 
                              steps_per_epoch=len(train_generator), 
                              validation_data=validation_generator, 
                              validation_steps=len(validation_generator), 
                              epochs=1, 
                              verbose=1,
                              callbacks=callbacks)

In [ ]:
#Bước 10: Load lại model:
model = load_model("../input/latest-model/latest_model.h5")

In [ ]:
#Bước 10.5: Lưu mô hình 
model.save("latest_model1.h5")

In [ ]:
#Bước 11: Chuẩn bị dữ liệu thử nghiệm
test_link="./test1"
test_filenames = os.listdir(test_link)

In [ ]:
#Bước 12: Dự đoán kết quả trên 1 số ảnh

dog_counter = 0 
cat_counter  = 0

for file in test_filenames:
    img = image.load_img(test_link + "/" + file, target_size=(224,224))
    x = cv2.imread(test_link + "/" + file)
    x = cv2.resize(x, Image_Size)
   
    classes = model.predict(np.expand_dims(x, axis=0))[0]
        
    if classes == 0:
      plt.xlabel("cat")
      plt.imshow(img)
      plt.show()
      cat_counter += 1
    else:
      plt.xlabel("dog")
      plt.imshow(img)
      plt.show()
      dog_counter += 1
        
    if(cat_counter==10): break

      
print("Total Dogs :",dog_counter)
print("Total Cats :",cat_counter)

In [ ]:
#Bước 13: Test 1 ảnh truyền vào - CHẠY NẾU MUỐN TEST 1 ẢNH = TAY
def predict_(link):
    image = cv2.imread(link)
    image = cv2.resize(image, (224,224))
    preds = model.predict(np.expand_dims(image, axis=0))[0]
    plt.imshow(image)
    plt.show()
    if preds==0:
            print("Predicted Label:Cat")
    else:
            print("Predicted Label: Dog")
        
        
predict_("../input/testcase/con_meo_cua_truc.jpg")

In [ ]:
#Bước 14: Đánh giá tỉ lệ chính xác - SAU KHI TRAIN N EPOCHS (KHÔNG DÙNG NẾU LOAD MODEL)
import sys

# Plot diagnostic learning curves
def summarize_diagnostics(history):

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

# Learning curves
summarize_diagnostics(history)